In [7]:
%%writefile src/pipeline_loop_unroll.cpp
#include <CL/sycl.hpp>
#include <iostream>
#include <sycl/exit/intel/fpga_extensions.hpp>
#include <chrono>
using namespace sycl;


// Forward declare the kernel name in the global scope.
// This FPGA best practice reduces name mangling in the optimization reports.
template <int unroll_factor> class Pipeline;

// This function instantiates the vector pipeline kernel, which contains
// a loop that peforms the pipeline operations on the two summand arrays and stores the result
// into output. This loop will be unrolled by the specified unroll_factor.
template <int unroll_factor>
void VecAdd(const std::vector<float> &arr1,
            const std::vector<float> &arr2, std::vector<float> &output,
            size_t array_size) {


#if defined(FPGA_EMULATOR)
  ext::intel::fpga_emulator_selector device_selector;
#elif defined(FPGA_SIMULATOR)
  ext::intel::fpga_simulator_selector device_selector;
#else
  ext::intel::fpga_selector device_selector;
#endif

  try {
    queue q(device_selector, fpga_tools::exception_handler,
            property::queue::enable_profiling{});

    //Declaring buffers to manage data between host and device 
    buffer buffer_arr1(arr1);
    buffer buffer_arr2(arr2);
    buffer buffer_out(output);

    event e = q.submit([&](handler &h) {
      //Different accessors for each buffer. Note the read and write properties
      accessor acc_arr1(buffer_arr1, h, read_only);
      accessor acc_arr2(buffer_arr2, h, read_only);
      accessor acc_out(buffer_out, h, write_only, no_init);

      h.single_task<Pipeline<unroll_factor>>([=]()
                                         [[intel::kernel_args_restrict]] {
        // Unroll the loop fully or partially, depending on unroll_factor
        #pragma unroll unroll_factor
        for (size_t i = 0; i < array_size; i++) {
          //Example pipeline code. Note that the pipeline does not have any stall depedencies 
          acc_out[i] = (acc_arr1[i]*acc_arr2[i]) + acc_arr1[i] ;
        }
      });
    });

    double start = e.get_profiling_info<info::event_profiling::command_start>();
    double end = e.get_profiling_info<info::event_profiling::command_end>();
    // convert from nanoseconds to ms
    double kernel_time = (double)(end - start) * 1e-6;

    std::cout << "unroll_factor " << unroll_factor
              << " kernel time : " << kernel_time << " ms\n";
    std::cout << "Throughput for kernel with unroll_factor " << unroll_factor
              << ": ";
    std::cout << std::fixed << std::setprecision(3)
#if defined(FPGA_SIMULATOR)
              << ((double)array_size / kernel_time) / 1e3f << " MFlops\n";
#else
              << ((double)array_size / kernel_time) / 1e6f << " GFlops\n";
#endif

  } catch (sycl::exception const &e) {
    // Catches exceptions in the host code
    std::cerr << "Caught a SYCL host exception:\n" << e.what() << "\n";

    // Most likely the runtime couldn't find FPGA hardware!
    if (e.code().value() == CL_DEVICE_NOT_FOUND) {
      std::cerr << "If you are targeting an FPGA, please ensure that your "
                   "system has a correctly configured FPGA board.\n";
      std::cerr << "Run sys_check in the oneAPI root directory to verify.\n";
      std::cerr << "If you are targeting the FPGA emulator, compile with "
                   "-DFPGA_EMULATOR.\n";
    }
    std::terminate();
  }
}

int main(int argc, char *argv[]) {
#if defined(FPGA_SIMULATOR)
  size_t array_size = 1 << 10;
#else
  size_t array_size = 1 << 26;
#endif

  if (argc > 1) {
    std::string option(argv[1]);
    if (option == "-h" || option == "--help") {
      std::cout << "Usage: \n<executable> <data size>\n\nFAILED\n";
      return 1;
    } else {
      array_size = std::stoi(option);
    }
  }

  std::vector<float> arr1(array_size);
  std::vector<float> arr2(array_size);

  std::vector<float> output_unrollx1(array_size);
  std::vector<float> output_unrollx2(array_size);
  std::vector<float> output_unrollx4(array_size);
  std::vector<float> output_unrollx8(array_size);
  std::vector<float> output_unrollx16(array_size);

  // Initialize the two arrays 
  for (size_t i = 0; i < array_size; i++) {
    arr1[i] = static_cast<float>(i + 1);
    arr2[i] = static_cast<float>(array_size - i);
  }

  std::cout << "Input Array Size:  " << array_size << "\n";

  // Instantiate Pipeline kernel with different unroll factors: 1, 2, 4, 8, 16
  VecAdd<1>(arr1, arr2, output_unrollx1, array_size);
  VecAdd<2>(arr1, arr2, output_unrollx2, array_size);
  VecAdd<4>(arr1, arr2, output_unrollx4, array_size);
  VecAdd<8>(arr1, arr2, output_unrollx8, array_size);
  VecAdd<16>(arr1, arr2, output_unrollx16, array_size);

  // Verify that the output data is the same for every unroll factor
  return 0;
}

Overwriting src/pipeline_loop_unroll.cpp


In [8]:
%%writefile src/pipeline_loop_unroll.sh
#!/bin/bash
source /opt/intel/inteloneapi/setvars.sh > /dev/null 2>&1

echo ====================
echo pipeline loop unroll
dpcpp src/pipeline_loop_unroll.cpp -o src/pipeline_loop_unroll -w -O3
src/pipeline_loop_unroll
echo ====================

Writing src/pipeline_loop_unroll.sh


In [9]:
%%writefile src/submit_job.sh
#==========================================
# Copyright © 2020 Intel Corporation
#
# SPDX-License-Identifier: MIT
#==========================================
# Script to submit job in Intel(R) DevCloud
# Version: 0.72
#==========================================

if [ -z "$1" ]; then
    echo "Missing script argument, Usage: ./q run.sh"
elif [ ! -f "$1" ]; then
    echo "File $1 does not exist"
else
    echo "Job has been submitted to Intel(R) DevCloud and will execute soon."
    echo ""
    script=$1
    property=$2
     if [ "$property" == "GPU GEN9" ]; then
             value="gen9"   
        elif [ "$property" == "GPU Iris XE Max" ]; then
            value="iris_xe_max"
        elif [ "$property" == "CPU Xeon 8153" ]; then
            value="renderkit"
        elif [ "$property" == "CPU Xeon 8256" ]; then
            value="stratix10"
        elif [ "$property" == "CPU Xeon 6128" ]; then
            value="skl"
        else
            value="gen9" 
    fi
    if [ "$property" == "{device.value}" ]; then
        echo "Selected Device is: GPU"
    else
        echo "Selected Device is: "$property
    fi
    echo ""
    # Remove old output files
    rm *.sh.* > /dev/null 2>&1
    # Submit job using qsub
    qsub_id=`qsub -l nodes=1:$value:ppn=2 -d . $script`
    job_id="$(cut -d'.' -f1 <<<"$qsub_id")"
    # Print qstat output
    qstat 
    # Wait for output file to be generated and display
    echo ""
    echo -ne "Waiting for Output "
    until [ -f $script.o$job_id ]; do
        sleep 1
        echo -ne "█"
        ((timeout++))
        # Timeout if no output file generated within 60 seconds
        if [ $timeout == 60 ]; then
            echo ""
            echo ""
            echo "TimeOut 60 seconds: Job is still queued for execution, check for output file later ($script.o$job_id)"
            echo ""
            break
        fi
    done
    # Print output and error file content if exist
    if [ -n "$(find -name '*.sh.o'$job_id)" ]; then
        echo " Done⬇"
        cat $script.o$job_id
        cat $script.e$job_id
        echo "Job Completed in $timeout seconds."
        rm *.sh.*$job_id > /dev/null 2>&1
    fi
fi

Writing src/submit_job.sh


In [10]:
! chmod 755 src/submit_job.sh; chmod 755 src/pipeline_loop_unroll.sh; src/submit_job.sh src/pipeline_loop_unroll.sh "GPU Gen9";

Job has been submitted to Intel(R) DevCloud and will execute soon.

Selected Device is: GPU Gen9

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2039868.v-qsvr-1           ...ub-singleuser u177643         00:00:25 R jupyterhub     
2039885.v-qsvr-1           ...oop_unroll.sh u177643                0 Q batch          

Waiting for Output ████████████████████████████████████████████████████████████

TimeOut 60 seconds: Job is still queued for execution, check for output file later (src/pipeline_loop_unroll.sh.o2039885)

 Done⬇
cat: src/pipeline_loop_unroll.sh.o2039885: No such file or directory
cat: src/pipeline_loop_unroll.sh.e2039885: No such file or directory
Job Completed in 60 seconds.
